In [15]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

## FewShotPromptTemplate 실습을 위해 import
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

## LengthBasedExampleSelector 실습을 위해 import
from langchain.prompts.example_selector import LengthBasedExampleSelector

## Cache 사용을 위해 import
## SQLiteCache는 DB기반 캐싱
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

from langchain.callbacks import StreamingStdOutCallbackHandler

## 캐싱 사용 설정
## InMemoryCache 사용할 경우
## set_llm_cache(InMemoryCache())
## DB Cache 사용할 경우
set_llm_cache(SQLiteCache("cache.db"))


## debug 설정
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,     ##Streaming 옵션 ON
    callbacks=[StreamingStdOutCallbackHandler()]
)

## Template은 Disk에 Save/Load되기 때문에 이런 목적으로도 사용
## large model 생성 시 Disk load
## 정공법 코딩은 아래와 같음
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)

t.format(country="Korea")


'What is the capital of Korea'

## FewShotPromptTemplate
답변에 대한 구체적인 요구사항이 필요할 경우, Prompt에서 줄글로 입력하는 것보다   
몇 개의 예제를 제공해(Few-shot) 학습시키는 것이 더욱 효과적임

Few-shot을 위해서는 답변 예제의 형식화가 필요함
이를 위해서는 형식 지정 도구가 필요. 아래 예제의 경우 직접 작성했지만 일반적으로 database에서 가져올 것임.

아래 예제에서 question에 대한 형식을 3개 주었고,
answer에 대한 형식도 3개 주었음. answer의 경우 Capital, Language, Food, ... 로 형식화하여 주었음.

In [16]:
## 특정 국가에 대해 무엇을 아나요? 라는 질문에 대한 examples (few-shot)
examples = [
{
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
},
{
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
},
{
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

##Example Seletor
example_selector=LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,  ## 이 값에 따라 예제의 양을 정해줄 수 있음
)

## FewShotPrompt 에 example을 set
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    ##examples=examples,  --- 위의 selector 사용을 위해 주석처리
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?", ##형식화된 모든 예제 마지막에 나오는 내용 (질문)
    input_variables=["country"], ## 어떤 변수를 suffix에서 사용할 것인지 
)

prompt.format(country="Germany")



'\n    Human: What do you know about France?\n    AI: \n    Here is what I know:\n    Capital: Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n    \n\n\nHuman: What do you know about Germany?'

## FewShotChatMessagePromptTemplate

In [16]:
"""

example_prompt2 = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}"), 
])

example_prompt2 = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt2,
    examples=examples2
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt2,
    ("human", "What do you know about {country}?"),
])

chain2 = final_prompt | chat
chain2.invoke({
    "country":"Germany"
})
"""


    I know this:
    Capital: Berlin
    Language: German
    Food: Bratwurst and Sauerkraut
    Currency: Euro

AIMessageChunk(content='\n    I know this:\n    Capital: Berlin\n    Language: German\n    Food: Bratwurst and Sauerkraut\n    Currency: Euro')

## LengthBasedExampleSelector

example의 동적 선택 지원 (어느 정도의 example을 선택해 prompt에 허용할 것인지를 결정)   
예제 형식화 및 그 양이 얼마나 되는지 확인 가능 + 설정값에 따라 prompt에 알맞은 예제를 골라 줌   
이 selector의 경우 말 그대로 예제의 length를 설정하고, 그 값을 기준으로 example을 선택 (허용량 한계치로 이해)   
유저의 로그인 여부 또는 유저가 사용하는 언어에 따라 example들을 얼마나 허용할 지 정할 수 있음   

### 이 외에도...
- Similarity 기반
- MMR (Max Marginal Relevance) 기반
- N-gram 기반

In [28]:
### randomExampleSelector
from typing import Any, Dict
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_selector_r = RandomExampleSelector(
    examples=examples,
)

prompt_r = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector_r,
    suffix="Human: What do you know about {country}?", ##형식화된 모든 예제 마지막에 나오는 내용 (질문)
    input_variables=["country"], ## 어떤 변수를 suffix에서 사용할 것인지 
)

prompt_r.format(country="Brazil")

'Human:What do you know about Greece?\nAI:\n    I know this:\n    Capital: Athens\n    Language: Greek\n    Food: Souvlaki and Feta Cheese\n    Currency: Euro\n    \n\nHuman: What do you know about Brazil?'

In [4]:
## 파일을 통해 prompt 가져오기
from langchain.prompts import load_prompt

## json / yaml prompt load
prompt_ser = load_prompt("./prompt.yaml")

prompt_ser.format(country="Germany")


## 여러 prompt를 합칠 수 있도록 하는 Tempalte
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# final prompt는 intro, example, start를 합친 Template임.
final = PromptTemplate.from_template(
    ## 이 예제에서는 role playing을 주제로 하였으며,
    ## intro : AI에게 부여할 role에 대한 Template
    ## example : 대화에 대한 예시 Template (kind of Few-shot)
    ## start : AI 도우미가 text를 완성시켜 주는 Template
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

## prompt
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final,)


'What is the capital of Germany'

## Caching
predict 때마다 답변을 생성하지 않고, cache 를 통해 답변속도를 향상

1. InMemoryCache
2. 

In [14]:
chat.predict("How do you make italian pasta")
## 1회차 : 3.4s
## 2회차 : 0.0s
## 같은 질문에 대해 답변을 재활용하여 효율 증대


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- 1/2 teaspoon of salt
- Water (if needed)

Here is a step-by-step guide on how to make Italian pasta:

1. On a clean work surface, pour the flour and make a well in the center.
2. Crack the eggs into the well and add the salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.
7. Cut the pasta sheet into your desired shape, such as fettuccine, spaghetti, 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the pasta sheet into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. D

suffix : 예제를 던진 뒤에 나오는 진짜 궁금한 질문 (이 예시처럼 내 질문에 대답해줘)